# 深層学習ノートブック-6 誤差逆伝播(Backpropagation)
誤差逆伝播をスクラッチで実装してみる   
* $l$番目の層における損失の誤差 
$$\delta^{[l]}=\frac{\partial{L}}{\partial{Z^{[l]}}}$$  
$$=\frac{\partial{L}}{\partial{A^{[l]}}} \bigodot \frac{\partial{A}}{\partial{Z^{[l]}}}$$
$$=(\delta^{[l+1]}\bm{W}^{[l+1]}) \bigodot \sigma'(\bm{Z^{[l]}})$$  

* パラメタの勾配
$$\frac{\partial{L}}{\partial{W^{[l]}}}=\delta^{[l]T}\bm{A}^{[l-1]}$$  
$$\frac{\partial{L}}{\partial{b^{[l]}}}=\sum_i\delta_i^{[l]}$$  

* パラメタの更新
$$\bm{W^{[l]}}=\bm{W^{[l]}}-\alpha\frac{\partial{L}}{\partial{W^{[l]}}}$$  
$$\bm{B^{[l]}}=\bm{B^{[l]}}-\alpha\frac{\partial{L}}{\partial{B^{[l]}}}$$  


※ReLUの導関数
\begin{equation}
\sigma^{'}(z)= \left \{
\begin{array}{l}
1　(if z > 0) \\
0　(otherwise)
\end{array}
\right.
\end{equation}

In [1]:
import torch
import torch.nn.functional as F  #pytorchの便利関数はFでimportすることが多い。
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# python debugerをインポート
import pdb

In [7]:
# 線形変換部分(パラメタ)の勾配の算出関数。Aはl-1層目の値で他はl層目の値。
def linear_backward(A, W, b, Z):
     # pytorch.tensorの.grad属性と区別するために.grad_として属性を定義している
     W.grad_ = Z.grad_ @ A
     # bの値が一つ変わると、全データ分Lへ影響を及ぼすので、行方向(データ数の方向)にZ.grad_を足す。
     b.grad_ = torch.sum(Z.grad_, dim=0)
     # l-1層目のAはl層目の誤差（=Z.grad_）とl層目のWから計算できる。
     A.grad_ = Z.grad_ @ W


# ReLU関数
def relu_backward(Z, A):
     # 上式の"l番目の層における損失の誤差"における2行目と3行目を組み合わせてdL/dZを算出している
     # A.grad_はdL/dA, (Z > 0).float()はReLu関数の偏微分に相当する。
     Z.grad_ = A.grad_ * ( Z > 0 ).float()

## 補足
tensor * (tensor > 0)の計算について

In [20]:
a = torch.randn((2,3)) 
b = torch.randn((2,3)) 
print(a)
print(b)
print(b > 0)

tensor([[ 0.8516, -1.0509,  0.1294],
        [-0.1999,  1.1394,  1.4127]])
tensor([[ 0.1299,  0.2585, -1.2203],
        [ 0.6629, -0.2176, -1.2055]])
tensor([[ True,  True, False],
        [ True, False, False]])


In [21]:
a * (b > 0)

tensor([[ 0.8516, -1.0509,  0.0000],
        [-0.1999,  0.0000,  0.0000]])

上記のようにb > 0がTrueの要素は1、Falseの要素は0として計算される。